1. Category analysis.
2. The best time to push vedio.
3. The correlation among views, likes, dislikes and comment_count.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Import dataset

In [ ]:
us_videos = pd.read_csv('../input/youtube-new/USvideos.csv')
us_videos_categories = pd.read_json('../input/youtube-new/US_category_id.json')

In [ ]:
us_videos.head()

In [ ]:
us_videos.trending_date.unique()

In [ ]:
us_videos_categories.head()

In [ ]:
us_videos_categories['items'][1]

Add category column

us_videos dataframe has a column categroy_id, by using this we could join us_videos_categories and add a new 'category' column which has category title 

In [ ]:
us_videos.info()

In [ ]:
#us_videos = us_videos.drop(columns=['category'])

id_to_title = {}

for item in us_videos_categories['items']:
    id_to_title[item['id']] = item['snippet']['title']
    
us_videos.insert(5, 'category', us_videos['category_id'].astype(str).map(id_to_title))

us_videos.head()

> **Top popular trending category**

In [ ]:
category_video_count = us_videos.groupby(["category"]).size().sort_values(ascending = False)
category_video_count

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(category_video_count.values, category_video_count.index.values, palette = "rocket")
plt.title("Top popular trending categories")
plt.xlabel("Video Count")
plt.show()


Add like/dislike ratio column

**Top view categories**

In [ ]:
category_views = us_videos.groupby(["category"]).mean().sort_values(by = "views", ascending = False)
sns.barplot(category_views["views"], category_views.index.values, palette = "rocket")
plt.title("Top Viewed Categories")
plt.xlabel("Average Views")
plt.show()

Top like ratio categories

In [ ]:
us_videos['like_to_dislike'] = us_videos['likes'] / us_videos['dislikes']

In [ ]:
category_like_ratio = us_videos[us_videos["dislikes"] != 0].groupby("category").mean().sort_values(by = "like_to_dislike", ascending = False)

sns.barplot(category_like_ratio["like_to_dislike"], category_like_ratio.index.values, palette = "rocket")

plt.title("Top Liked Categories")
plt.xlabel("Average Like to Dislike Ratio")
plt.show()

Publish time

In [ ]:
# Transforming Trending date column to datetime format
us_videos['trending_date'] = pd.to_datetime(us_videos['trending_date'], format='%y.%d.%m').dt.date

# Transforming Trending date column to datetime format and splitting into two separate ones
publish_time = pd.to_datetime(us_videos['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
us_videos['publish_date'] = publish_time.dt.date
us_videos['publish_time'] = publish_time.dt.time
us_videos['publish_hour'] = publish_time.dt.hour

us_videos.head()

In [ ]:
publish_hour_data = us_videos.groupby(["publish_hour"]).size()
publish_hour_data

In [ ]:
plt.plot(publish_hour_data.index.values, publish_hour_data.values)
plt.scatter(publish_hour_data.index.values, publish_hour_data.values)
plt.xlabel("Hour of Day")
plt.ylabel("Video Count")
plt.title("Trending Vedio Published by the Hour")
plt.show()

In [ ]:
publish_hour_data_average = us_videos.groupby(["publish_hour"]).mean()
plt.plot(publish_hour_data_average.index.values, publish_hour_data_average["views"])
plt.scatter(publish_hour_data_average.index.values, publish_hour_data_average["views"])
plt.xlabel("Hour of Day")
plt.ylabel("Average Number of Views")
plt.title("Average Amount of Views on Trending Videos by the Hour")
plt.show()

In [ ]:
plt.plot(publish_hour_data_average.index.values, publish_hour_data_average["likes"])
plt.scatter(publish_hour_data_average.index.values, publish_hour_data_average["likes"])
plt.xlabel("Hour of Day")
plt.ylabel("Average Number of likes")
plt.title("Average Amount of likes on Trending Videos by the Hour")
plt.show()

**The correaltion between views, likes, dislikes and comment_count**

In [ ]:
correlation_list = ['views', 'likes', 'dislikes', 'comment_count', 'like_to_dislike']

correlation = us_videos[correlation_list].corr()

display(correlation)

In [ ]:
sns.heatmap(correlation)